In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Load and preprocess your data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'sorted_training/train',
    labels='inferred',
    label_mode='categorical',
    class_names=['1', '2', '3', '4', '5', '6', '7', '8', '9'],
    image_size=(30,48),
    batch_size=32)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'sorted_training/test',
    labels='inferred',
    label_mode='categorical',
    class_names=['1', '2', '3', '4', '5', '6', '7', '8', '9'],
    image_size=(30,48),
    batch_size=32)

# Build your model
model = keras.Sequential([
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(9, activation='softmax')
])

# Compile your model
model.compile(
    optimizer='adam',
    loss=tf.losses.CategoricalCrossentropy(),
    metrics=['accuracy'])

# Train your model
model.fit(train_ds, epochs=15)

# Evaluate your model
loss, accuracy = model.evaluate(test_ds)
print(f'Test accuracy: {accuracy}')


Found 1172 files belonging to 9 classes.
Found 385 files belonging to 9 classes.
Epoch 1/15
37/37 [==============================] - 2s 25ms/step - loss: 1.5383 - accuracy: 0.5102
Epoch 2/15
37/37 [==============================] - 1s 26ms/step - loss: 0.3888 - accuracy: 0.8882
Epoch 3/15
37/37 [==============================] - 1s 25ms/step - loss: 0.2001 - accuracy: 0.9411
Epoch 4/15
37/37 [==============================] - 1s 25ms/step - loss: 0.1515 - accuracy: 0.9548
Epoch 5/15
37/37 [==============================] - 1s 25ms/step - loss: 0.1081 - accuracy: 0.9718
Epoch 6/15
37/37 [==============================] - 1s 24ms/step - loss: 0.0797 - accuracy: 0.9761
Epoch 7/15
37/37 [==============================] - 1s 24ms/step - loss: 0.0725 - accuracy: 0.9787
Epoch 8/15
37/37 [==============================] - 1s 23ms/step - loss: 0.0615 - accuracy: 0.9795
Epoch 9/15
37/37 [==============================] - 1s 25ms/step - loss: 0.0456 - accuracy: 0.9872
Epoch 10/15
37/37 [=========

In [27]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Define the path to the image you want to make a prediction for
image_path = 'sorted_training/test/4/captcha_0.jpg_1_digit4.jpg'

# Load the image and preprocess it
img = image.load_img(image_path, target_size=(30, 48))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

# Use the trained model to make a prediction
prediction = model.predict(x)
predicted_class = np.argmax(prediction[0])

# Print the predicted class label
print(f'Predicted class: {predicted_class}')


1/1 [==============================] - 0s 23ms/step
Predicted class: 3
